# Effective Range Expansion plot

Notebook which plots Effective range exapansion by reading in files

## Imports

In [ ]:
from itertools import product

import numpy as np
import pandas as pd

from matplotlib.pylab import plt
import seaborn as sns

from luescher_nd.zeta.zeta3d import Zeta3D
from luescher_nd.zeta.extern.pyzeta import zeta as zeta_cont

from luescher_nd.utilities import get_laplace_coefficients

from sympy import S

In [ ]:
%load_ext blackcellmagic

## Parameters & Files

In [ ]:
HBARC = 197.326 # MeV / fm
M_NUCLEON = (938.27 + 939.57) / 2 / HBARC # in fm^{-1}

mu = M_NUCLEON / 2

In [ ]:
FILEREAD = "../data/luescher-3d-res-mom-nstep-inf.csv"
FILEREAD = "../data/luescher-3d-dispersion-final.csv"

df = pd.read_csv(FILEREAD, dtype={"nstep": "Int64"})

df.loc[df.nstep.isna(), "nstep"] = -1

df["L"] = df["n1d_max"] * df["epsilon"]
df["x"] = df["energy"] * mu * df["L"] ** 2 / 2 / np.pi ** 2

df["y"] = np.nan
df["continuum"] = False

for L, epsilon, nstep in product(df.L.unique(), df.epsilon.unique(), df.nstep.unique()):
    ind = (df.L == L) & (df.epsilon == epsilon) & (df.nstep == nstep)

    nstep = nstep if nstep > 0 else None
    zeta = Zeta3D(L, epsilon, nstep=nstep)
    free_x = np.unique(zeta._get_n2())
    df.loc[ind, "continuum"] = df.loc[ind, "x"].apply(
        lambda val: np.abs(val - free_x).min() < 0.05
    )
    df.loc[ind, "y"] = zeta(df.loc[ind, "x"].values) / np.pi / L

df["nstep"] = df.apply(
    lambda row: "$\infty$" if row["nstep"] < 0 else f"${row['nstep']}$", axis=1
)

df = df[df.continuum == False]
df = df.drop_duplicates(["epsilon", "nstep", "y"]).dropna()

df = df.sort_values(["x", "n1d_max", "nstep"], ascending=[True, True, True])

df.head()


## Plot

In [ ]:
ymax = 0.5

g = sns.FacetGrid(
    df,
    col="nstep",
    hue="n1d_max",
    hue_kws={"marker": ["s", "o", "v", "d"]},
    palette="cubehelix",
    xlim=(-0.1, 19),
    #ylim=(-0.01, ymax),
    col_order=sorted(df.nstep.unique())
)

g.map(plt.plot, "x", "y", ms=1, lw=0.5, ls="--")

g.set_xlabels("$x$")
g.set_ylabels(r"$\frac{S_3^{Lat}(x)}{\pi L}$")

g.add_legend(title=r"$\frac{L}{\epsilon}$", fontsize=6)

x = np.linspace(-1, 20, 1000)

L = 1.0
epsilon= 0.1

zeta = Zeta3D(L, epsilon)
for ax in g.axes[0]:
    ax.axhline(0, c="black", lw=0.2)
    ax.axvline(0, c="black", lw=0.2)
    ax.set_yscale("log")
    #ax.plot(x, zeta(x)/np.pi/L, "--", lw=0.1, ms=0.1)
    
g.fig.set_dpi(300)
g.fig.set_figheight(2)
g.fig.set_figwidth(6)

plt.show(g.fig)